In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200


In [2]:
def get_tables(tables):
    df = pd.DataFrame()
    for tb in tables:

        if type(tb.columns) == pd.core.indexes.multi.MultiIndex:
            cols = [col[0] for col in tb.columns]
            tb.columns = cols

        if "TOTALIZAÇÃO" not in tb.columns:
            tipo = tb.columns[0]
            unnamed_col = [col for col in tb if "Unnamed" in col]
            unnamed = unnamed_col[0] if unnamed_col != [] else ""

            if unnamed != "":
                tb = tb[tb[unnamed].isnull()]
                tb = tb.drop(unnamed_col, 1)
            tb["TIPO"] = tipo
            tb = tb.rename(columns={tipo: "ESCOLAS"})
            df = pd.concat([df, tb])

    cols = ["ESCOLAS", "JURADO 01", "JURADO 02", "JURADO 03", "JURADO 04", "TIPO"]
    for col in cols:
        if col not in df.columns:
            df[col] = np.nan

    return df[cols]


def pos_treatment(df):

    cols = ["escola", "jurado_1", "jurado_2", "jurado_3", "jurado_4", "tipo", "ano"]
    df.columns = cols

    cols = ["ano", "escola", "tipo", "jurado_1", "jurado_2", "jurado_3", "jurado_4"]
    df = df[cols]

    mask = df["jurado_1"].notnull()
    df = df[mask]

    mask = df["jurado_1"].str.contains("Jurado")
    df = df[np.logical_not(mask)]

    cols = ["jurado_1", "jurado_2", "jurado_3", "jurado_4"]
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


In [3]:
anos = list(range(1968, 2021)) + list(range(2022,2023))
df_final = pd.DataFrame()

for ano in anos:
    url = f"https://galeriadosamba.com.br/carnaval/{ano}/notas/"
    print(ano, url)
    try:
        tables = pd.read_html(url)
        dd = get_tables(tables)
        dd["ano"] = ano
        df_final = pd.concat([df_final, dd])
    except Exception:
        pass

df = pos_treatment(df_final)

1968 https://galeriadosamba.com.br/carnaval/1968/notas/
1969 https://galeriadosamba.com.br/carnaval/1969/notas/
1970 https://galeriadosamba.com.br/carnaval/1970/notas/
1971 https://galeriadosamba.com.br/carnaval/1971/notas/
1972 https://galeriadosamba.com.br/carnaval/1972/notas/
1973 https://galeriadosamba.com.br/carnaval/1973/notas/
1974 https://galeriadosamba.com.br/carnaval/1974/notas/
1975 https://galeriadosamba.com.br/carnaval/1975/notas/
1976 https://galeriadosamba.com.br/carnaval/1976/notas/
1977 https://galeriadosamba.com.br/carnaval/1977/notas/
1978 https://galeriadosamba.com.br/carnaval/1978/notas/
1979 https://galeriadosamba.com.br/carnaval/1979/notas/
1980 https://galeriadosamba.com.br/carnaval/1980/notas/
1981 https://galeriadosamba.com.br/carnaval/1981/notas/
1982 https://galeriadosamba.com.br/carnaval/1982/notas/
1983 https://galeriadosamba.com.br/carnaval/1983/notas/
1984 https://galeriadosamba.com.br/carnaval/1984/notas/
1985 https://galeriadosamba.com.br/carnaval/1985

In [77]:
# df.to_csv('historico_notas_carnaval_rio.csv', index=False)
df = pd.read_csv('historico_notas_carnaval_rio.csv')

In [82]:
cols = ["jurado_1", "jurado_2", "jurado_3", "jurado_4"]

df["menor_nota"] = np.sort(df[cols], axis=1)[:, :1].T[0] * -1
df["nota_final"] = df[cols + ["menor_nota"]].sum(axis=1)

df[["ano", "escola", "nota_final"]].groupby(
    ["escola", "ano"], as_index=False
).sum().sort_values(by=["ano", "nota_final"], ascending=False).drop_duplicates("ano")



# Esses dados nao estao fazendo muito sentido, com repeticao de notas a partir de 2018
# alem de aparentemente as notas contabilizacao da nota final mudou em algum periodo

,escola,ano,nota_final
154,Beija-Flor de Nilópolis,2022,269.6
153,Beija-Flor de Nilópolis,2020,269.6
152,Beija-Flor de Nilópolis,2019,269.6
151,Beija-Flor de Nilópolis,2018,269.6
495,Portela,2017,269.9
237,Estação Primeira de Mangueira,2016,269.8
270,Estácio de Sá,2015,299.7
713,Unidos do Viradouro,2014,299.9
652,Unidos de Vila Isabel,2013,299.7
582,Unidos da Tijuca,2012,299.9
